## Reading of ASCII files created for cam diagnostics tool

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd
from glob import glob
import os
import helper_funcs as helpers
import ipywidgets as ipw

### 1. Paths and global settings (GLOB)  

Please change accordingly if you execute this notebook on your local machine.

#### 1.1. Paths (PATHS)

Here you can specify your paths.

In [2]:
#folder with ascii files
data_dir = "./data/michael_ascii_read/"
file_type = "webarchive"

# file containing additional information about variables (long names, can be interactively updated below)
varinfo_csv = "./data/var_info.csv"

# Config file for different groups
vargroups_cfg = "./data/varconfig.ini"

#directy to store results
output_dir = "./output/"

#### Global settings (SETUP)

In the following cells you can specify global default settings.

##### Define group of variables that you are interested in

Default group of variables. Variable groups can be defined in [varconfig.ini](https://github.com/jgliss/my_notebooks/blob/master/data/varconfig.ini). Use ``[group_name]`` to define a new group and add below all variables that should belong to the group in the desired display order (should be self-explanatory when looking at the file, I hope).

In [3]:
var_group = None #group_name from varconfig.ini (use None, if you want to use all)

##### Add data columns to index

Use the following list to specify table columns that should be added to the multiindex (Ada, here is where you can add "Obs").

In [4]:
add_to_index = None #["Obs"]

##### Define which parts of index should be unstacked

The following list can be used to specify how the final lists are displayed. The items in the list need to be names of sub-indices in the the Multiindex of the originally loaded file (i.e. "Run", "Years", "Variable", "Description") or data columns that were added to index (previous option). 

All values specified here will be unstacked, i.e. put from the original row into a column index representation (makes table view wider).

In [5]:
unstack_indices = ["Run", "Years"]

##### Shortcuts for Run IDs

Define list of shortnames for model runs or define a prefix. If undefined (i.e. empty list and ``None``), the original names are used.

In [6]:
#either
run_ids = list("ABC")
#or
run_id_prefix = "Run" #None

### 2. Importing and editing supplementary information

Let's begin with reading the variable information from the excel table. Note that this is not strictly required but helps us below to display the results in a more intuitive manner, when analysing the data.

Note that the following method makes sure the CSV file exists, i.e. if it has not been created before, the information is loaded from Michaels Excel table and then saved at ``varinfo_csv``.

In [7]:
var_info_dict = helpers.load_varinfo(varinfo_csv)

The following cell opens an interactive widget that can be used to edit the information available for each variable (stored in file ``varinfo_csv``, see previous cell).

In [8]:
from my_widgets import EditDictCSV

edit_config = EditDictCSV(varinfo_csv)
#show
edit_config()

Now update to the current selection (run everything below if you change the previous cell).

In [9]:
var_info_dict = edit_config.var_dict

### 3. Search and load ASCII files, either using .asc or .webarchive file type (LOAD_FILE)

The following cell finds all files in folder ``data_dir``.

In [10]:
files = sorted(glob(data_dir + "*.{}".format(file_type)))
for file in files:
    print(file)    

./data/michael_ascii_read/N1850C53CLM45L32_f09_tn11_191017 (yrs 71-100).webarchive
./data/michael_ascii_read/N1850_f09_tn14_230218 (yrs 1-20).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 185-215).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 310-340).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 80-110).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r265_ctrl_20180411 (yrs 90-120).webarchive


### 4. Importing multiple result files and concatenating them into one Dataframe (LOAD_FILES)

In the following, we load all files into one `Dataframe`. 

To do this, a custom method `read_and_merge_all` was defined in [helper_funcs.py](https://github.com/jgliss/my_py3_scripts/blob/master/notebooks/helper_funcs.py). The method basically loops over all files and calls the method ``read_file_custom``, which you can also find in [helper_funcs.py](https://github.com/jgliss/my_py3_scripts/blob/master/notebooks/helper_funcs.py). 

In [11]:
merged = helpers.read_and_merge_all(file_list=files, var_info_dict=var_info_dict, replace_runid_prefix=run_id_prefix)
if add_to_index:
    for item in add_to_index:
        merged = merged.set_index([merged.index, item])
merged

/home/jonasg/github/my_notebooks/helper_funcs.py:99: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.test_case = pd.Series(mapping)


Flag     Model       Obs  \
Run  Years  Variable           Description                                      
Run1 71-100 RESTOM             TOmodel net flux      True    -0.489     0.000   
            RESSURF            SRF net flux          True    -0.489     0.000   
            RESTOA_CERES-EBAF  TOA  net flux         True     1.529     0.992   
            RESTOA_ERBE                             False     1.529     0.059   
            SOLIN_CERES-EBAF                        False   340.206   340.054   
            SOLIN_CERES                             False   340.206   341.479   
            CLDTOT_ISCCP       Total cloud cover     True    63.621    66.800   
            CLDTOT_CLOUDSAT                         False    63.621    66.824   
            FLDS_ISCCP         LW down SRF           True   338.280   343.347   
            FLNS_ISCCP         LW net SRF            True    55.819    49.425   
            FLUT_CERES-EBAF    LW up Top             True   238.148   239.574   
            FLUT_CERES                              False   238.148   238.963   
            FLUT_ERBE                               False   238.148   233.946   
            FLUTC_CERES-EBAF   LW up Top Clearsky    True   261.783   266.051   
            FLUTC_CERES                             False   261.783   266.878   
            FLUTC_ERBE                              False   261.783   264.429   
            FLNT_CAM                                False   236.838       NaN   
            FSDS_ISCCP         SW down SRF           True   187.801   189.390   
            FSNS_ISCCP         SW net SRF            True   163.679   165.893   
            FSNS_LARYEA                             False   163.679   154.089   
            FSNTOA_CERES-EBAF                       False   239.677   240.566   
            FSNTOA_CERES       SW net TOA            True   239.677   244.691   
            FSNTOA_ERBE                             False   239.677   234.004   
            FSNTOAC_CERES-EBAF                      False   287.999   287.638   
            FSNTOAC_CERES      SW net TOA clearsky   True   287.999   294.702   
            FSNTOAC_ERBE                            False   287.999   289.329   
            FSNT_CAM                                False   236.348       NaN   
            LHFLX_JRA25        Lat Heat Flux         True    87.904    87.935   
            LHFLX_ERA40                             False    87.904    82.367   
            LHFLX_WHOI                              False    87.904    89.083   
...                                                   ...       ...       ...   
Run4 90-120 PRECT_GPCP         Precipitation         True     2.918     2.674   
            PREH2O_NVAP                             False    25.733    24.597   
            PREH2O_AIRS                             False    25.733    20.425   
            PREH2O_JRA25                            False    25.733    20.425   
            PREH2O_ERAI                             False    25.733    24.247   
            PREH2O_ERA40       Precipitable water    True    25.733    24.948   
            PSL_JRA25          SeaLev pressure       True  1011.614  1011.280   
            PSL_ERAI                                False  1011.614  1011.362   
            SHFLX_JRA25        Sens Heat Flux        True    22.229    19.372   
            SHFLX_NCEP                              False    22.229    15.795   
            SHFLX_LARYEA                            False    22.229    13.236   
            STRESS_MAG_ERS                          False     0.063     0.064   
            STRESS_MAG_LARYEA                       False     0.063     0.071   
            STRESS_MAG_JRA25                        False     0.063     0.080   
            SWCF_CERES-EBAF    SW Cloud Forc         True   -48.959   -47.071   
            SWCF_CERES                              False   -48.959   -48.586   
            SWCF_ERBE                               False   -48.959   -54.163   
   

### 5. Rearranging and restructuring of the imported data (REARRANGE)

In the following cell, you can interacively select which Variables you wish to keep for further analysis. Preselected are the variables that are flagged.

#### 5.1 Interactive selection of variables (IA_VAR)

In [12]:
from my_widgets import SelectVariable    
selector = SelectVariable(df=merged, level="Variable", preconfig_file=vargroups_cfg,
                         default_group=var_group)
#show
selector()

Now access the current selection and continue.

In [13]:
selection = selector.df_edit
selection

Flag     Model       Obs  \
Run  Years  Variable          Description                                     
Run1 71-100 RESTOM            TOmodel net flux     True    -0.489     0.000   
            RESSURF           SRF net flux         True    -0.489     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     1.529     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    63.621    66.800   
            FLDS_ISCCP        LW down SRF          True   338.280   343.347   
            FLNS_ISCCP        LW net SRF           True    55.819    49.425   
            FLUT_CERES-EBAF   LW up Top            True   238.148   239.574   
            FLUTC_CERES-EBAF  LW up Top Clearsky   True   261.783   266.051   
            FSDS_ISCCP        SW down SRF          True   187.801   189.390   
            FSNS_ISCCP        SW net SRF           True   163.679   165.893   
            FSNTOA_CERES      SW net TOA           True   239.677   244.691   
            FSNTOAC_CERES     SW net TOA clearsky  True   287.999   294.702   
            LHFLX_JRA25       Lat Heat Flux        True    87.904    87.935   
            LWCF_CERES-EBAF   LW Cloud Forc        True    23.635    26.476   
            PRECT_GPCP        Precipitation        True     3.003     2.674   
            PREH2O_ERA40      Precipitable water   True    23.974    24.948   
            PSL_JRA25         SeaLev pressure      True  1011.281  1011.280   
            SHFLX_JRA25       Sens Heat Flux       True    20.445    19.372   
            SWCF_CERES-EBAF   SW Cloud Forc        True   -48.323   -47.071   
            SST_HADISST_PI    SST preindustrial    True    20.180    20.288   
            TREFHT_JRA25      2m Temp              True   286.507   287.605   
            TS_NCEP           2m Temp              True   287.337   287.683   
            TS_LAND_NCEP      2m Temp land         True   280.631   281.155   
            U_200_JRA25       Zonal Wind JRA25     True    16.571    15.565   
            Z3_500_JRA25      Geop Height  JRA25   True    56.347    56.601   
Run2 1-20   RESTOM            TOmodel net flux     True     0.987     0.000   
            RESSURF           SRF net flux         True     0.974     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     3.075     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    68.586    66.800   
            FLDS_ISCCP        LW down SRF          True   341.547   343.347   
...                                                 ...       ...       ...   
Run3 80-110 TREFHT_JRA25      2m Temp              True   288.036   287.605   
            TS_NCEP           2m Temp              True   289.247   287.683   
            TS_LAND_NCEP      2m Temp land         True   283.336   281.155   
            U_200_JRA25       Zonal Wind JRA25     True    18.905    15.565   
            Z3_500_JRA25      Geop Height  JRA25   True    56.714    56.601   
Run4 90-120 RESTOM            TOmodel net flux     True    -0.075     0.000   
            RESSURF           SRF net flux         True    -0.075     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     2.013     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    70.747    66.800   
            FLDS_ISCCP        LW down SRF          True   347.854   343.347   
            FLNS_ISCCP        LW net SRF           True    56.350    49.425   
            FLUT_CERES-EBAF   LW up Top            True   238.742   239.574   
            FLUTC_CERES-EBAF  LW up Top Clearsky   True   264.869   266.051   
            FSDS_ISCCP        SW down SRF          True   187.669   189.390   
            FSNS_ISCCP        SW net SRF           True   163.747   165.893   
            FSNTOA_CERES      SW net TOA           True   240.755   244.691   
            FSNTOAC_CERES     SW net TOA clearsky  True   289.714   294.702   
            LHFLX_JRA25       Lat Heat Flux        True    85.243    87.935   
            LWCF_CERES

#### 5.2 Interactive index renaming tool (IA_RENAME)

In the following, an interactive widget is defined, that allows for renaming of the runs.

In [14]:
from my_widgets import IndexRenamer

renamer = IndexRenamer(selection, suggestions=run_ids)
renamer()

Now, update the current dataframe for further usage.

In [15]:
selection = renamer.df_edit
selection

Flag     Model       Obs  \
Run  Years  Variable          Description                                     
A    71-100 RESTOM            TOmodel net flux     True    -0.489     0.000   
            RESSURF           SRF net flux         True    -0.489     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     1.529     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    63.621    66.800   
            FLDS_ISCCP        LW down SRF          True   338.280   343.347   
            FLNS_ISCCP        LW net SRF           True    55.819    49.425   
            FLUT_CERES-EBAF   LW up Top            True   238.148   239.574   
            FLUTC_CERES-EBAF  LW up Top Clearsky   True   261.783   266.051   
            FSDS_ISCCP        SW down SRF          True   187.801   189.390   
            FSNS_ISCCP        SW net SRF           True   163.679   165.893   
            FSNTOA_CERES      SW net TOA           True   239.677   244.691   
            FSNTOAC_CERES     SW net TOA clearsky  True   287.999   294.702   
            LHFLX_JRA25       Lat Heat Flux        True    87.904    87.935   
            LWCF_CERES-EBAF   LW Cloud Forc        True    23.635    26.476   
            PRECT_GPCP        Precipitation        True     3.003     2.674   
            PREH2O_ERA40      Precipitable water   True    23.974    24.948   
            PSL_JRA25         SeaLev pressure      True  1011.281  1011.280   
            SHFLX_JRA25       Sens Heat Flux       True    20.445    19.372   
            SWCF_CERES-EBAF   SW Cloud Forc        True   -48.323   -47.071   
            SST_HADISST_PI    SST preindustrial    True    20.180    20.288   
            TREFHT_JRA25      2m Temp              True   286.507   287.605   
            TS_NCEP           2m Temp              True   287.337   287.683   
            TS_LAND_NCEP      2m Temp land         True   280.631   281.155   
            U_200_JRA25       Zonal Wind JRA25     True    16.571    15.565   
            Z3_500_JRA25      Geop Height  JRA25   True    56.347    56.601   
B    1-20   RESTOM            TOmodel net flux     True     0.987     0.000   
            RESSURF           SRF net flux         True     0.974     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     3.075     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    68.586    66.800   
            FLDS_ISCCP        LW down SRF          True   341.547   343.347   
...                                                 ...       ...       ...   
C    80-110 TREFHT_JRA25      2m Temp              True   288.036   287.605   
            TS_NCEP           2m Temp              True   289.247   287.683   
            TS_LAND_NCEP      2m Temp land         True   283.336   281.155   
            U_200_JRA25       Zonal Wind JRA25     True    18.905    15.565   
            Z3_500_JRA25      Geop Height  JRA25   True    56.714    56.601   
Run4 90-120 RESTOM            TOmodel net flux     True    -0.075     0.000   
            RESSURF           SRF net flux         True    -0.075     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     2.013     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    70.747    66.800   
            FLDS_ISCCP        LW down SRF          True   347.854   343.347   
            FLNS_ISCCP        LW net SRF           True    56.350    49.425   
            FLUT_CERES-EBAF   LW up Top            True   238.742   239.574   
            FLUTC_CERES-EBAF  LW up Top Clearsky   True   264.869   266.051   
            FSDS_ISCCP        SW down SRF          True   187.669   189.390   
            FSNS_ISCCP        SW net SRF           True   163.747   165.893   
            FSNTOA_CERES      SW net TOA           True   240.755   244.691   
            FSNTOAC_CERES     SW net TOA clearsky  True   289.714   294.702   
            LHFLX_JRA25       Lat Heat Flux        True    85.243    87.935   
            LWCF_CERES

#### 5.3 Reshaping of table (make it wider for readibility) (RESHAPE)
 
For visualisation this display requires a lot of scrolling. We can make the table `wider` by unstacking certain indices, e.g. the two outermost indices `Run` and `Years`.

In [16]:
selection_unstacked = selection.unstack(unstack_indices)
selection_unstacked

Flag                               \
Run                                        A     B       C                  
Years                                 71-100  1-20 185-215 310-340 80-110   
Variable          Description                                               
CLDTOT_ISCCP      Total cloud cover     True  True    True    True   True   
FLDS_ISCCP        LW down SRF           True  True    True    True   True   
FLNS_ISCCP        LW net SRF            True  True    True    True   True   
FLUTC_CERES-EBAF  LW up Top Clearsky    True  True    True    True   True   
FLUT_CERES-EBAF   LW up Top             True  True    True    True   True   
FSDS_ISCCP        SW down SRF           True  True    True    True   True   
FSNS_ISCCP        SW net SRF            True  True    True    True   True   
FSNTOAC_CERES     SW net TOA clearsky   True  True    True    True   True   
FSNTOA_CERES      SW net TOA            True  True    True    True   True   
LHFLX_JRA25       Lat Heat Flux         True  True    True    True   True   
LWCF_CERES-EBAF   LW Cloud Forc         True  True    True    True   True   
PRECT_GPCP        Precipitation         True  True    True    True   True   
PREH2O_ERA40      Precipitable water    True  True    True    True   True   
PSL_JRA25         SeaLev pressure       True  True    True    True   True   
RESSURF           SRF net flux          True  True    True    True   True   
RESTOA_CERES-EBAF TOA  net flux         True  True    True    True   True   
RESTOM            TOmodel net flux      True  True    True    True   True   
SHFLX_JRA25       Sens Heat Flux        True  True    True    True   True   
SST_HADISST_PI    SST preindustrial     True  True    True    True   True   
SWCF_CERES-EBAF   SW Cloud Forc         True  True    True    True   True   
TREFHT_JRA25      2m Temp               True  True    True    True   True   
TS_LAND_NCEP      2m Temp land          True  True    True    True   True   
TS_NCEP           2m Temp               True  True    True    True   True   
U_200_JRA25       Zonal Wind JRA25      True  True    True    True   True   
Z3_500_JRA25      Geop Height  JRA25    True  True    True    True   True   

                                                 Model                      \
Run                                     Run4         A         B         C   
Years                                 90-120    71-100      1-20   185-215   
Variable          Description                                                
CLDTOT_ISCCP      Total cloud cover     True    63.621    68.586    68.543   
FLDS_ISCCP        LW down SRF           True   338.280   341.547   353.861   
FLNS_ISCCP        LW net SRF            True    55.819    59.124    56.249   
FLUTC_CERES-EBAF  LW up Top Clearsky    True   261.783   265.065   267.090   
FLUT_CERES-EBAF   LW up Top             True   238.148   241.278   241.972   
FSDS_ISCCP        SW down SRF           True   187.801   192.590   190.458   
FSNS_ISCCP        SW net SRF            True   163.679   168.522   166.962   
FSNTOAC_CERES     SW net TOA clearsky   True   287.999   289.651   290.330   
FSNTOA_CERES      SW net TOA            True   239.677   244.353   244.525   
LHFLX_JRA25       Lat Heat Flux         True    87.904    85.432    87.926   
LWCF_CERES-EBAF   LW Cloud Forc         True    23.635    23.787    25.119   
PRECT_GPCP        Precipitation         True     3.003     2.920     3.010   
PREH2O_ERA40      Precipitable water    True    23.974    24.437    28.091   
PSL_JRA25         SeaLev pressure       True  1011.281  1011.284  1011.771   
RESSURF           SRF net flux          True    -0.489     0.974     0.474   
RESTOA_CERES-EBAF TOA  net flux         True     1.529     3.075     2.553   
RESTOM            TOmodel net flux      True    -0.489     0.987     0.468   
SHFLX_JRA25       Sens Heat Flux        True    20.445    22.993    22.314   
SST_HADISST_PI    SST preindustrial     True    20.180    21.075    22.582   
SWCF_CERES-E

Well, this is better but also not extremely illustrative / intuitive. It becomes more intuitive if we just look at one parameter that we are interested in (e.g. RMSE). 

#### 5.4 Extracting the Bias of each model run relative to the observations (GET_BIAS)

Retrieving a table that illustrates the Bias of each run for each flagged variable is straight forward. We just extract the `Bias` column from our flagged frame:

In [17]:
bias = selection_unstacked["Bias"]
bias.head()

Run                                      A      B       C                 \
Years                               71-100   1-20 185-215 310-340 80-110   
Variable         Description                                               
CLDTOT_ISCCP     Total cloud cover  -3.179  1.786   1.744   1.435  2.157   
FLDS_ISCCP       LW down SRF        -5.066 -1.799  10.514  11.499  5.162   
FLNS_ISCCP       LW net SRF          6.394  9.699   6.824   6.847  7.167   
FLUTC_CERES-EBAF LW up Top Clearsky -4.268 -0.986   1.039   1.426 -0.786   
FLUT_CERES-EBAF  LW up Top          -1.426  1.704   2.398   2.928  0.660   

Run                                   Run4  
Years                               90-120  
Variable         Description                
CLDTOT_ISCCP     Total cloud cover   3.947  
FLDS_ISCCP       LW down SRF         4.507  
FLNS_ISCCP       LW net SRF          6.925  
FLUTC_CERES-EBAF LW up Top Clearsky -1.182  
FLUT_CERES-EBAF  LW up Top          -0.832

#### 5.5 Computing RMSE relative error (GET_RMSE_REL)

In the following we extract the subset containing the *RSME* information of the flagged variables for all runs in order to compute the relative error for each run based on the average *RMSE* of all runs:

$$\frac{RMSE_{Run}\,-\,\overline{RMSE_{All\,Runs}}}{\overline{RMSE_{All\,Runs}}}$$


In [18]:
rmse = selection_unstacked["RMSE"]
rmse

Run                                         A       B       C                  \
Years                                  71-100    1-20 185-215 310-340  80-110   
Variable          Description                                                   
CLDTOT_ISCCP      Total cloud cover    11.323  11.881  12.992  13.078  12.869   
FLDS_ISCCP        LW down SRF          14.450  15.351  16.891  17.664  16.720   
FLNS_ISCCP        LW net SRF           11.967  14.953  14.098  14.174  14.516   
FLUTC_CERES-EBAF  LW up Top Clearsky    6.042   4.778   4.662   4.738   5.670   
FLUT_CERES-EBAF   LW up Top             6.855   6.169   7.188   7.467   7.499   
FSDS_ISCCP        SW down SRF          13.380  15.089  15.915  16.082  16.048   
FSNS_ISCCP        SW net SRF           12.711  12.632  13.587  13.727  13.705   
FSNTOAC_CERES     SW net TOA clearsky  18.458  17.432  15.977  15.710  18.506   
FSNTOA_CERES      SW net TOA           12.307  10.795  12.096  12.125  12.314   
LHFLX_JRA25       Lat Heat Flux        17.176  14.578  14.947  15.116  15.587   
LWCF_CERES-EBAF   LW Cloud Forc         6.156   5.683   5.860   5.980   6.018   
PRECT_GPCP        Precipitation         1.162   0.891   1.043   1.069   1.055   
PREH2O_ERA40      Precipitable water    2.147   1.920   4.153   4.430   3.351   
PSL_JRA25         SeaLev pressure       1.333   1.512   1.825   1.939   1.940   
RESSURF           SRF net flux            NaN     NaN     NaN     NaN     NaN   
RESTOA_CERES-EBAF TOA  net flux         8.842   8.358   9.011   8.937   8.872   
RESTOM            TOmodel net flux        NaN     NaN     NaN     NaN     NaN   
SHFLX_JRA25       Sens Heat Flux       11.761  11.926  11.051  11.138  11.551   
SST_HADISST_PI    SST preindustrial     1.059   1.223   2.458   2.606   2.135   
SWCF_CERES-EBAF   SW Cloud Forc        10.324   8.854  10.447  10.563  10.393   
TREFHT_JRA25      2m Temp               2.316   1.955   2.538   2.634   2.665   
TS_LAND_NCEP      2m Temp land          2.767   3.132   5.068   5.229   4.164   
TS_NCEP           2m Temp               1.990   2.320   3.382   3.529   3.004   
U_200_JRA25       Zonal Wind JRA25      2.444   2.852   4.334   4.348   4.335   
Z3_500_JRA25      Geop Height  JRA25    0.280   0.192   0.391   0.423   0.335   

Run                                      Run4  
Years                                  90-120  
Variable          Description                  
CLDTOT_ISCCP      Total cloud cover    12.485  
FLDS_ISCCP        LW down SRF          15.278  
FLNS_ISCCP        LW net SRF           13.988  
FLUTC_CERES-EBAF  LW up Top Clearsky    4.778  
FLUT_CERES-EBAF   LW up Top             6.598  
FSDS_ISCCP        SW down SRF          15.421  
FSNS_ISCCP        SW net SRF           13.068  
FSNTOAC_CERES     SW net TOA clearsky  17.609  
FSNTOA_CERES      SW net TOA           12.711  
LHFLX_JRA25       Lat Heat Flux        15.153  
LWCF_CERES-EBAF   LW Cloud Forc         5.673  
PRECT_GPCP        Precipitation         0.975  
PREH2O_ERA40      Precipitable water    2.373  
PSL_JRA25         SeaLev pressure       1.695  
RESSURF           SRF net flux            NaN  
RESTOA_CERES-EBAF TOA  net flux         9.827  
RESTOM            TOmodel net flux        NaN  
SHFLX_JRA25       Sens Heat Flux       11.202  
SST_HADISST_PI    SST preindustrial     1.732  
SWCF_CERES-EBAF   SW Cloud Forc         9.973  
TREFHT_JRA25      2m Temp               2.147  
TS_LAND_NCEP      2m Temp land          3.944  
TS_NCEP           2m Temp               2.663  
U_200_JRA25       Zonal Wind JRA25      3.342  
Z3_500_JRA25      Geop Height  JRA25    0.199

##### Side comment: Series vs. unstacked Multiindex Dataframes

As you can see in the previous output, we have extracted ***ONE*** variable from the ***UNSTACKED*** dataframe. Now, this is still a pandas ``Dataframe`` since it is *tabular* data. 

In [19]:
print("Extracted table is Dataframe since it is a wide table: {}".format(isinstance(rmse, pd.DataFrame)))

Extracted table is Dataframe since it is a wide table: True


In [20]:
rmse_mean = rmse.mean(axis=1, skipna=True)
#Note that the created object is a Series and not a Dataframe
rmse_mean.head()

Variable          Description       
CLDTOT_ISCCP      Total cloud cover     12.438000
FLDS_ISCCP        LW down SRF           16.059000
FLNS_ISCCP        LW net SRF            13.949333
FLUTC_CERES-EBAF  LW up Top Clearsky     5.111333
FLUT_CERES-EBAF   LW up Top              6.962667
dtype: float64

The next step is (semi) straight forward (we have to use the `div` and `subtract` methods of the Dataframe rather than `/` and `-` operators in order to specify that we want to apply them in the horizontal and not in the vertical direction.

In [21]:
rmse_err_rel = rmse.subtract(rmse_mean, axis=0).div(rmse_mean, axis=0)
rmse_err_rel

Run                                           A         B         C            \
Years                                    71-100      1-20   185-215   310-340   
Variable          Description                                                   
CLDTOT_ISCCP      Total cloud cover   -0.089645 -0.044782  0.044541  0.051455   
FLDS_ISCCP        LW down SRF         -0.100193 -0.044087  0.051809  0.099944   
FLNS_ISCCP        LW net SRF          -0.142110  0.071951  0.010658  0.016106   
FLUTC_CERES-EBAF  LW up Top Clearsky   0.182079 -0.065215 -0.087909 -0.073040   
FLUT_CERES-EBAF   LW up Top           -0.015463 -0.113989  0.032363  0.072434   
FSDS_ISCCP        SW down SRF         -0.126774 -0.015239  0.038669  0.049568   
FSNS_ISCCP        SW net SRF          -0.039834 -0.045801  0.026338  0.036913   
FSNTOAC_CERES     SW net TOA clearsky  0.068048  0.008680 -0.075512 -0.090962   
FSNTOA_CERES      SW net TOA           0.020650 -0.104744  0.003151  0.005556   
LHFLX_JRA25       Lat Heat Flux        0.113433 -0.054982 -0.031062 -0.020107   
LWCF_CERES-EBAF   LW Cloud Forc        0.044275 -0.035963 -0.005937  0.014419   
PRECT_GPCP        Precipitation        0.125424 -0.137046  0.010169  0.035351   
PREH2O_ERA40      Precipitable water  -0.298901 -0.373027  0.356155  0.446609   
PSL_JRA25         SeaLev pressure     -0.219250 -0.114408  0.068918  0.135689   
RESSURF           SRF net flux              NaN       NaN       NaN       NaN   
RESTOA_CERES-EBAF TOA  net flux       -0.014764 -0.068695  0.004067 -0.004179   
RESTOM            TOmodel net flux          NaN       NaN       NaN       NaN   
SHFLX_JRA25       Sens Heat Flux       0.028224  0.042650 -0.033849 -0.026243   
SST_HADISST_PI    SST preindustrial   -0.433336 -0.345581  0.315259  0.394453   
SWCF_CERES-EBAF   SW Cloud Forc        0.022955 -0.122700  0.035142  0.046636   
TREFHT_JRA25      2m Temp             -0.025184 -0.177131  0.068257  0.108664   
TS_LAND_NCEP      2m Temp land        -0.316903 -0.226794  0.251152  0.290899   
TS_NCEP           2m Temp             -0.292989 -0.175746  0.201563  0.253790   
U_200_JRA25       Zonal Wind JRA25    -0.322835 -0.209790  0.200831  0.204710   
Z3_500_JRA25      Geop Height  JRA25  -0.076923 -0.367033  0.289011  0.394505   

Run                                                  Run4  
Years                                    80-110    90-120  
Variable          Description                              
CLDTOT_ISCCP      Total cloud cover    0.034652  0.003779  
FLDS_ISCCP        LW down SRF          0.041161 -0.048633  
FLNS_ISCCP        LW net SRF           0.040623  0.002772  
FLUTC_CERES-EBAF  LW up Top Clearsky   0.109300 -0.065215  
FLUT_CERES-EBAF   LW up Top            0.077030 -0.052375  
FSDS_ISCCP        SW down SRF          0.047349  0.006428  
FSNS_ISCCP        SW net SRF           0.035251 -0.012867  
FSNTOAC_CERES     SW net TOA clearsky  0.070825  0.018921  
FSNTOA_CERES      SW net TOA           0.021231  0.054155  
LHFLX_JRA25       Lat Heat Flux        0.010426 -0.017708  
LWCF_CERES-EBAF   LW Cloud Forc        0.020865 -0.037659  
PRECT_GPCP        Precipitation        0.021792 -0.055690  
PREH2O_ERA40      Precipitable water   0.094264 -0.225101  
PSL_JRA25         SeaLev pressure      0.136275 -0.007224  
RESSURF           SRF net flux              NaN       NaN  
RESTOA_CERES-EBAF TOA  net flux       -0.011421  0.094991  
RESTOM            TOmodel net flux          NaN       NaN  
SHFLX_JRA25       Sens Heat Flux       0.009865 -0.020647  
SST_HADISST_PI    SST preindustrial    0.142424 -0.073219  
SWCF_CERES-EBAF   SW Cloud Forc        0.029792 -0.011824  
TREFHT_JRA25      2m Temp              0.121712 -0.096317  
TS_LAND_NCEP      2m Temp land         0.027979 -0.026333  
TS_NCEP           2m Temp              0.067267 -0.053884  
U_200_JRA25       Zonal Wind JRA25     0.201108 -0.074024  
Z3_500_JRA25      Geop Height  JRA25   0.104396 -0.343956

### 5.6 Inserting column of RMSE relative error into original table (INSERT_RMSE_REL_ORIG)

If we want, we can now add the typical RMSE to our original dataframe (containing the only flagged data, since it was computed from this). 

**Note: this is just illustrative and not used in the following section**

First we have to stack it:

In [22]:
stacked = rmse_err_rel.stack(level=(0,1)).reorder_levels(order=(2,3,0,1))
stacked.head()

Run  Years    Variable      Description      
A    71-100   CLDTOT_ISCCP  Total cloud cover   -0.089645
B    1-20     CLDTOT_ISCCP  Total cloud cover   -0.044782
C    185-215  CLDTOT_ISCCP  Total cloud cover    0.044541
     310-340  CLDTOT_ISCCP  Total cloud cover    0.051455
     80-110   CLDTOT_ISCCP  Total cloud cover    0.034652
dtype: float64

In [23]:
selection["RMSE_ERR"] = stacked
selection

Flag     Model       Obs  \
Run  Years  Variable          Description                                     
A    71-100 RESTOM            TOmodel net flux     True    -0.489     0.000   
            RESSURF           SRF net flux         True    -0.489     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     1.529     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    63.621    66.800   
            FLDS_ISCCP        LW down SRF          True   338.280   343.347   
            FLNS_ISCCP        LW net SRF           True    55.819    49.425   
            FLUT_CERES-EBAF   LW up Top            True   238.148   239.574   
            FLUTC_CERES-EBAF  LW up Top Clearsky   True   261.783   266.051   
            FSDS_ISCCP        SW down SRF          True   187.801   189.390   
            FSNS_ISCCP        SW net SRF           True   163.679   165.893   
            FSNTOA_CERES      SW net TOA           True   239.677   244.691   
            FSNTOAC_CERES     SW net TOA clearsky  True   287.999   294.702   
            LHFLX_JRA25       Lat Heat Flux        True    87.904    87.935   
            LWCF_CERES-EBAF   LW Cloud Forc        True    23.635    26.476   
            PRECT_GPCP        Precipitation        True     3.003     2.674   
            PREH2O_ERA40      Precipitable water   True    23.974    24.948   
            PSL_JRA25         SeaLev pressure      True  1011.281  1011.280   
            SHFLX_JRA25       Sens Heat Flux       True    20.445    19.372   
            SWCF_CERES-EBAF   SW Cloud Forc        True   -48.323   -47.071   
            SST_HADISST_PI    SST preindustrial    True    20.180    20.288   
            TREFHT_JRA25      2m Temp              True   286.507   287.605   
            TS_NCEP           2m Temp              True   287.337   287.683   
            TS_LAND_NCEP      2m Temp land         True   280.631   281.155   
            U_200_JRA25       Zonal Wind JRA25     True    16.571    15.565   
            Z3_500_JRA25      Geop Height  JRA25   True    56.347    56.601   
B    1-20   RESTOM            TOmodel net flux     True     0.987     0.000   
            RESSURF           SRF net flux         True     0.974     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     3.075     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    68.586    66.800   
            FLDS_ISCCP        LW down SRF          True   341.547   343.347   
...                                                 ...       ...       ...   
C    80-110 TREFHT_JRA25      2m Temp              True   288.036   287.605   
            TS_NCEP           2m Temp              True   289.247   287.683   
            TS_LAND_NCEP      2m Temp land         True   283.336   281.155   
            U_200_JRA25       Zonal Wind JRA25     True    18.905    15.565   
            Z3_500_JRA25      Geop Height  JRA25   True    56.714    56.601   
Run4 90-120 RESTOM            TOmodel net flux     True    -0.075     0.000   
            RESSURF           SRF net flux         True    -0.075     0.000   
            RESTOA_CERES-EBAF TOA  net flux        True     2.013     0.992   
            CLDTOT_ISCCP      Total cloud cover    True    70.747    66.800   
            FLDS_ISCCP        LW down SRF          True   347.854   343.347   
            FLNS_ISCCP        LW net SRF           True    56.350    49.425   
            FLUT_CERES-EBAF   LW up Top            True   238.742   239.574   
            FLUTC_CERES-EBAF  LW up Top Clearsky   True   264.869   266.051   
            FSDS_ISCCP        SW down SRF          True   187.669   189.390   
            FSNS_ISCCP        SW net SRF           True   163.747   165.893   
            FSNTOA_CERES      SW net TOA           True   240.755   244.691   
            FSNTOAC_CERES     SW net TOA clearsky  True   289.714   294.702   
            LHFLX_JRA25       Lat Heat Flux        True    85.243    87.935   
            LWCF_CERES

### 6. Conditional formatting of tables (Dataframes) (VISUALISE)

This section illustrates, how we can perform conditional formatting of the color tables. As discussed above, we can apply background colour gradients to the data. In the example above we had a multiindex data type specifying model run, year-range and variable in stacked format (long table) and the four data columns specifying results from model and observation as well as bias and RMSE. 

Now, in the following we illustrate how we can apply this colour highlighting for the two unstacked tables that we just created and that contain Bias and relative error. 

Starting with the Bias data, we show an example that does not work for our purposes (since it only allows for conditional formatting of either rows or columns.

#### 6.1 NOT how we want it (using the style method `background_gradient`) (VIS_WRONG)

The most straight forward example for conditional formatting of a Dataframe is shown in the following. In the example we use the `Bias` table and, similar to the example above, apply a value based colormap. Here, we use a *diverging colormap (bwr)* which has white as center color. Like in the example above, we use the style method `background_gradient` which can perform the formatting either in a **rowwise** or **columnwise** manner (using input argument `axis=1` or `axis=0`, respectively). 

Note, however, that this is not what we are aiming for in this example, rather, we want the colour formatting to be applied based on the values available the **whole table** and not individually for **columns** or **rows** (which is done in the next section). Nonetheless, in the cell below we show what we get if we use the method `backgroun_gradient`.  

Again, we use the `low` and `high` parameters to specify the colorrange that we use to map the values (see above).

In [24]:
bias.style.background_gradient(cmap="bwr", low=0.5, high=0.5, axis=1).highlight_null("white")

In [25]:
bias

Run                                        A      B       C                 \
Years                                 71-100   1-20 185-215 310-340 80-110   
Variable          Description                                                
CLDTOT_ISCCP      Total cloud cover   -3.179  1.786   1.744   1.435  2.157   
FLDS_ISCCP        LW down SRF         -5.066 -1.799  10.514  11.499  5.162   
FLNS_ISCCP        LW net SRF           6.394  9.699   6.824   6.847  7.167   
FLUTC_CERES-EBAF  LW up Top Clearsky  -4.268 -0.986   1.039   1.426 -0.786   
FLUT_CERES-EBAF   LW up Top           -1.426  1.704   2.398   2.928  0.660   
FSDS_ISCCP        SW down SRF         -1.589  3.200   1.068   1.228  1.216   
FSNS_ISCCP        SW net SRF          -2.214  2.630   1.070   1.365 -0.017   
FSNTOAC_CERES     SW net TOA clearsky -6.703 -5.051  -4.373  -4.183 -5.747   
FSNTOA_CERES      SW net TOA          -5.015 -0.338  -0.167   0.223 -1.724   
LHFLX_JRA25       Lat Heat Flux       -0.031 -2.503  -0.009   0.434 -1.695   
LWCF_CERES-EBAF   LW Cloud Forc       -2.841 -2.689  -1.357  -1.501 -1.446   
PRECT_GPCP        Precipitation        0.329  0.246   0.336   0.352  0.276   
PREH2O_ERA40      Precipitable water  -0.975 -0.511   3.143   3.437  1.922   
PSL_JRA25         SeaLev pressure      0.002  0.004   0.491   0.513  0.448   
RESSURF           SRF net flux        -0.489  0.974   0.474   0.337  0.643   
RESTOA_CERES-EBAF TOA  net flux        0.537  2.083   1.561   1.420  1.741   
RESTOM            TOmodel net flux    -0.489  0.987   0.468   0.327  0.647   
SHFLX_JRA25       Sens Heat Flux       1.074  3.621   2.942   2.907  3.028   
SST_HADISST_PI    SST preindustrial   -0.108  0.782   2.217   2.385  1.756   
SWCF_CERES-EBAF   SW Cloud Forc       -1.252  1.774   1.266   1.466  1.083   
TREFHT_JRA25      2m Temp             -1.098 -0.263   1.434   1.615  0.431   
TS_LAND_NCEP      2m Temp land        -0.524  1.237   3.765   3.945  2.181   
TS_NCEP           2m Temp             -0.346  0.876   2.579   2.767  1.564   
U_200_JRA25       Zonal Wind JRA25     1.006  1.825   3.209   3.218  3.340   
Z3_500_JRA25      Geop Height  JRA25  -0.254 -0.090   0.278   0.317  0.113   

Run                                     Run4  
Years                                 90-120  
Variable          Description                 
CLDTOT_ISCCP      Total cloud cover    3.947  
FLDS_ISCCP        LW down SRF          4.507  
FLNS_ISCCP        LW net SRF           6.925  
FLUTC_CERES-EBAF  LW up Top Clearsky  -1.182  
FLUT_CERES-EBAF   LW up Top           -0.832  
FSDS_ISCCP        SW down SRF         -1.721  
FSNS_ISCCP        SW net SRF          -2.145  
FSNTOAC_CERES     SW net TOA clearsky -4.988  
FSNTOA_CERES      SW net TOA          -3.937  
LHFLX_JRA25       Lat Heat Flux       -2.692  
LWCF_CERES-EBAF   LW Cloud Forc       -0.349  
PRECT_GPCP        Precipitation        0.244  
PREH2O_ERA40      Precipitable water   0.785  
PSL_JRA25         SeaLev pressure      0.335  
RESSURF           SRF net flux        -0.075  
RESTOA_CERES-EBAF TOA  net flux        1.021  
RESTOM            TOmodel net flux    -0.075  
SHFLX_JRA25       Sens Heat Flux       2.857  
SST_HADISST_PI    SST preindustrial    1.358  
SWCF_CERES-EBAF   SW Cloud Forc       -1.888  
TREFHT_JRA25      2m Temp              0.344  
TS_LAND_NCEP      2m Temp land         2.153  
TS_NCEP           2m Temp              1.518  
U_200_JRA25       Zonal Wind JRA25     2.304  
Z3_500_JRA25      Geop Height  JRA25  -0.014

Now, this worked nicely but there are mainly two problems with this representation:

1. As mentioned above, one problem here is that the colour coding can only be performed row or column wise using the input parameter `axis` (and not based on the values of the whole table, see [here](https://pandas.pydata.org/pandas-docs/stable/style.html#Building-Styles-Summary) for details)
2. If we use the symmetric colormap as is (i.e. center colour is white), then, the color white will be mapped to the midpoint value of the considered value range (e.g. min=-2, max=4 => (4 - -2)/2 = 3 => 1 == white). However, what we want is a *shifter diverging colormap* that ensures that the value 0 is mapped white, even if min != -max.
3. Further, we might wish to have control over the number of significant digits that are displayed in the table

All these problems will be solved in the following.

#### 6.2 How we want it (VIS_RIGHT)

In the following, we use a custom display method `my_table_display` (that is defined in [helper_funcs.py](https://github.com/jgliss/my_py3_scripts/blob/master/notebooks/helper_funcs.py)) in order to perform colour formatting considering all rows and columns at the same time and furthermore, using a diverging colour map that is dynamically shifted such that value 0 corresponds to the colour white (method `shifted_color_map`) also if `-vmin != vmax` (like usually).

In [26]:
from helper_funcs import my_table_display
my_table_display(bias)

Now for the typical RMSE error

In [27]:
my_table_display(rmse_err_rel)

/home/jonasg/anaconda3/lib/python3.6/site-packages/matplotlib/colors.py:489: RuntimeWarning: invalid value encountered in less
  np.copyto(xa, -1, where=xa < 0.0)


### 7. Concatenate and save results (Bias and typical RMSE) as table (EXPORT)

In the following, the two result tables ``bias_table`` and ``typical_rmse`` are merged into one result table and then saved both as excel table and as csv file.

In [28]:
result = pd.concat([bias, rmse_err_rel],axis=1, keys=["Bias", "RMSE relative Error"])
result

Bias                                \
Run                                        A      B       C                  
Years                                 71-100   1-20 185-215 310-340 80-110   
Variable          Description                                                
CLDTOT_ISCCP      Total cloud cover   -3.179  1.786   1.744   1.435  2.157   
FLDS_ISCCP        LW down SRF         -5.066 -1.799  10.514  11.499  5.162   
FLNS_ISCCP        LW net SRF           6.394  9.699   6.824   6.847  7.167   
FLUTC_CERES-EBAF  LW up Top Clearsky  -4.268 -0.986   1.039   1.426 -0.786   
FLUT_CERES-EBAF   LW up Top           -1.426  1.704   2.398   2.928  0.660   
FSDS_ISCCP        SW down SRF         -1.589  3.200   1.068   1.228  1.216   
FSNS_ISCCP        SW net SRF          -2.214  2.630   1.070   1.365 -0.017   
FSNTOAC_CERES     SW net TOA clearsky -6.703 -5.051  -4.373  -4.183 -5.747   
FSNTOA_CERES      SW net TOA          -5.015 -0.338  -0.167   0.223 -1.724   
LHFLX_JRA25       Lat Heat Flux       -0.031 -2.503  -0.009   0.434 -1.695   
LWCF_CERES-EBAF   LW Cloud Forc       -2.841 -2.689  -1.357  -1.501 -1.446   
PRECT_GPCP        Precipitation        0.329  0.246   0.336   0.352  0.276   
PREH2O_ERA40      Precipitable water  -0.975 -0.511   3.143   3.437  1.922   
PSL_JRA25         SeaLev pressure      0.002  0.004   0.491   0.513  0.448   
RESSURF           SRF net flux        -0.489  0.974   0.474   0.337  0.643   
RESTOA_CERES-EBAF TOA  net flux        0.537  2.083   1.561   1.420  1.741   
RESTOM            TOmodel net flux    -0.489  0.987   0.468   0.327  0.647   
SHFLX_JRA25       Sens Heat Flux       1.074  3.621   2.942   2.907  3.028   
SST_HADISST_PI    SST preindustrial   -0.108  0.782   2.217   2.385  1.756   
SWCF_CERES-EBAF   SW Cloud Forc       -1.252  1.774   1.266   1.466  1.083   
TREFHT_JRA25      2m Temp             -1.098 -0.263   1.434   1.615  0.431   
TS_LAND_NCEP      2m Temp land        -0.524  1.237   3.765   3.945  2.181   
TS_NCEP           2m Temp             -0.346  0.876   2.579   2.767  1.564   
U_200_JRA25       Zonal Wind JRA25     1.006  1.825   3.209   3.218  3.340   
Z3_500_JRA25      Geop Height  JRA25  -0.254 -0.090   0.278   0.317  0.113   

                                             RMSE relative Error            \
Run                                     Run4                   A         B   
Years                                 90-120              71-100      1-20   
Variable          Description                                                
CLDTOT_ISCCP      Total cloud cover    3.947           -0.089645 -0.044782   
FLDS_ISCCP        LW down SRF          4.507           -0.100193 -0.044087   
FLNS_ISCCP        LW net SRF           6.925           -0.142110  0.071951   
FLUTC_CERES-EBAF  LW up Top Clearsky  -1.182            0.182079 -0.065215   
FLUT_CERES-EBAF   LW up Top           -0.832           -0.015463 -0.113989   
FSDS_ISCCP        SW down SRF         -1.721           -0.126774 -0.015239   
FSNS_ISCCP        SW net SRF          -2.145           -0.039834 -0.045801   
FSNTOAC_CERES     SW net TOA clearsky -4.988            0.068048  0.008680   
FSNTOA_CERES      SW net TOA          -3.937            0.020650 -0.104744   
LHFLX_JRA25       Lat Heat Flux       -2.692            0.113433 -0.054982   
LWCF_CERES-EBAF   LW Cloud Forc       -0.349            0.044275 -0.035963   
PRECT_GPCP        Precipitation        0.244            0.125424 -0.137046   
PREH2O_ERA40      Precipitable water   0.785           -0.298901 -0.373027   
PSL_JRA25         SeaLev pressure      0.335           -0.219250 -0.114408   
RESSURF           SRF net flux        -0.075                 NaN       NaN   
RESTOA_CERES-EBAF TOA  net flux        1.021           -0.014764 -0.068695   
RESTOM            TOmodel net flux    -0.075                 NaN       NaN   
SHFLX_JRA25       Sens Heat Flux       2.857            0.028224  0.042650   
SST_HADISST_PI    SST preindustrial    1.358           -0.433

Now save both tables as excel file.

In [29]:
writer = pd.ExcelWriter('{}/result.xlsx'.format(output_dir))
result.to_excel(writer)
writer.save()